# STC Jawwy

In [ ]:
"""
Here we install libraries that are not installed by default
Example:  pyslsb
Feel free to add any library you are planning to use.
"""
!pip install pyxlsb

In [ ]:
# Import the required libraries
"""
Please feel free to import any required libraries as per your needs
"""
import pandas as pd     # provides high-performance, easy to use structures and data analysis tools
import pyxlsb           # Excel extention to read xlsb files (the input file)
import numpy as np      # provides fast mathematical computation on arrays and matrices

# Jawwy dataset
The dataset consists of meta details about the movies and tv shows as genre.
Also details about Users activities, spent duration and if watching in High definition or standard definition.
You have to analyse this dataset to find top insights, findings and to solve the four tasks assigned to you.

In [4]:
dataframe = pd.read_excel("/content/stc TV Data Set_T1.xlsb",sheet_name="Final_Dataset")
# Please make a copy of dataset if you are going to work directly and make changes on the dataset
# you can use   df=dataframe.copy()

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# check the data shape
dataframe.shape

(1048575, 13)

In [8]:
# display the first 5 rows
dataframe.head()

,Column1,date_,user_id_maped,program_name,duration_seconds,program_class,season,episode,program_desc,program_genre,series_title,hd,original_name
0,1,42882,26138,100 treets,40,MOVIE,0,0,Drama Movie100 Streets,Drama,0,0,100 treets
1,3,42876,7946,Moana,17,MOVIE,0,0,Animation MovieMoana (HD),Animation,0,1,Moana
2,4,42957,7418,The Mermaid Princess,8,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess
3,5,42942,19307,The Mermaid Princess,76,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess
4,7,42923,15860,Churchill,87,MOVIE,0,0,Biography MovieChurchill (HD),Biography,0,1,Churchill


In [12]:
# Data Preprocessing on the input data
# dataframe = dataframe.drop(columns=['Column1'])         # dropping the index column
dataframe['program_name'] = dataframe['program_name'].str.strip()  # trim spaces in movies names to avoid misspellings in input data
# dataframe['date_'] = pd.to_datetime(dataframe['date_'], unit='d', origin='1899-12-30')  # read date column as date data type
dataframe[['duration_seconds', 'season','episode','series_title','hd']] = dataframe[['duration_seconds', 'season','episode','series_title','hd']].apply(pd.to_numeric)  # read numeric columns as numeric data types
dataframe[['user_id_maped', 'program_name','program_class','program_desc','program_genre','original_name']] = dataframe[['user_id_maped', 'program_name','program_class','program_desc','program_genre','original_name']].astype(str) # read string columns as string data types

In [14]:
# display the dataset after applying data types
dataframe.head()

,date_,user_id_maped,program_name,duration_seconds,program_class,season,episode,program_desc,program_genre,series_title,hd,original_name
0,42882,26138,100 treets,40,MOVIE,0,0,Drama Movie100 Streets,Drama,0,0,100 treets
1,42876,7946,Moana,17,MOVIE,0,0,Animation MovieMoana (HD),Animation,0,1,Moana
2,42957,7418,The Mermaid Princess,8,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess
3,42942,19307,The Mermaid Princess,76,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess
4,42923,15860,Churchill,87,MOVIE,0,0,Biography MovieChurchill (HD),Biography,0,1,Churchill


In [15]:
# describe the numeric values in the dataset
dataframe.describe()

,date_,duration_seconds,season,episode,series_title,hd
count,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06
mean,4.301202e+04,1.230957e+03,1.342139e+00,6.157952e+00,1.205922e-02,3.862728e-01
std,1.242834e+02,6.821058e+03,2.104095e+00,1.222015e+01,1.091504e-01,4.868946e-01
min,4.280800e+04,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.289600e+04,5.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,4.302200e+04,1.190000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
75%,4.312100e+04,1.328000e+03,1.000000e+00,9.000000e+00,0.000000e+00,1.000000e+00
max,4.322000e+04,1.461329e+06,2.300000e+01,2.820000e+02,1.000000e+00,1.000000e+00


In [16]:
# check if any column has null value in the dataset
dataframe.isnull().any()

,0
date_,False
user_id_maped,False
program_name,False
duration_seconds,False
program_class,False
season,False
episode,False
program_desc,False
program_genre,False
series_title,False


# Task 1
##### You are required to work on task one to study and HD flag for available dataset

In [17]:
# make a copy of the dataframe for working on task 1
df=dataframe.copy()

In [18]:
# Here we try to get the most watched movies (Total Views / Total Users Views / Total watch time)
# For series we concatenated the Session episode to differentiate between episodes
grouped=df.copy()
grouped.loc[grouped['program_class'] == 'SERIES/EPISODES', 'program_name'] = grouped['program_name']+'_SE'+grouped['season'].astype(str)+'_EP'+grouped['episode'].astype(str)
grouped = grouped.groupby(['program_name','program_class'])\
.agg({'user_id_maped': [('co1', 'nunique'),('co2', 'count')],\
      'duration_seconds': [('co3', 'sum')] }).reset_index()
grouped.columns = ['program_name','program_class','No of Users who Watched', 'No of watches', 'Total watch time in seconds']
grouped['Total watch time in houres']=grouped['Total watch time in seconds']/3600
grouped = grouped.drop(columns=['Total watch time in seconds'])
grouped = grouped.sort_values(by=['Total watch time in houres', 'No of watches','No of Users who Watched'], ascending=False).reset_index(drop=True)


In [19]:
# show the result
grouped.head(35)

,program_name,program_class,No of Users who Watched,No of watches,Total watch time in houres
0,The Boss Baby,MOVIE,3389,24047,2961.350833
1,The Amazing pider-Man,MOVIE,1011,2877,1966.119167
2,The Expendables,MOVIE,853,2119,1961.159444
3,Moana,MOVIE,2173,8081,1706.176944
4,Trolls,MOVIE,2613,13793,1601.023056
5,Bean,MOVIE,949,3617,1423.955000
6,The murfs,MOVIE,867,3132,1342.141111
7,Hotel Transylvania,MOVIE,491,1947,1096.533611
8,Cloudy With a Chance of Meatballs,MOVIE,683,2076,948.674722
9,The Man With The Iron Fists,MOVIE,707,2505,859.626389


In [20]:
# we import Visualization libraries
# you can ignore and use any other graphing libraries
import matplotlib.pyplot as plt # a comprehensive library for creating static, animated, and interactive visualizations
import plotly #a graphing library makes interactive, publication-quality graphs. Examples of how to make line plots, scatter plots, area charts, bar charts, error bars, box plots, histograms, heatmaps, subplots, multiple-axes, polar charts, and bubble charts.
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [22]:
# plot top 10 Programs
fig = px.pie(grouped.head(10), values='Total watch time in houres', names='program_name',\
             hover_data=['program_class'],title='top 10 programs in total watch time in houres')
fig.show()

In [23]:
# Here we try to study the customer experience against Program class
grouped=df.copy()
grouped = grouped.groupby('program_class')\
.agg({'user_id_maped': [('co1', 'nunique'),('co2', 'count')],\
      'duration_seconds': [('co3', 'sum')] }).reset_index()
grouped.columns = ['program_class','No of Users who Watched', 'No of watches', 'Total watch time in seconds']
grouped['Total watch time in houres']=grouped['Total watch time in seconds']/3600
grouped = grouped.drop(columns=['Total watch time in seconds'])
grouped = grouped.sort_values(by=['Total watch time in houres', 'No of watches','No of Users who Watched'], ascending=False).reset_index(drop=True)


In [24]:
# show the result
grouped.head()

,program_class,No of Users who Watched,No of watches,Total watch time in houres
0,SERIES/EPISODES,3901,560174,255097.787500
1,MOVIE,11355,488401,103444.145556


In [25]:
# plot the total watch time against total number of users and report your findings
fig = px.pie(grouped, values='Total watch time in houres', names='program_class',\
             hover_data=['program_class'],title='Total duration spent by program_class')
fig2 = px.pie(grouped, values='No of Users who Watched', names='program_class',\
             hover_data=['program_class'],title='Total Users watching by program_class')

fig.update_traces(sort=False)
fig2.update_traces(sort=False)
fig.show()
fig2.show()

In [44]:
"""
TODO study the relation and user's behaviour Against HD flag, feel free to include any comparison
"""
df['quality_type'] = df['hd'].map({1: 'HD', 0: 'SD'})


hd_stats = df.groupby('quality_type').agg({
    'user_id_maped': ['nunique', 'count'],
    'duration_seconds': ['sum', 'mean']
}).reset_index()


hd_stats.columns = ['Quality', 'Unique_Users', 'Total_Views', 'Total_Duration_Seconds', 'Avg_Duration_Seconds']
hd_stats['Total_Duration_Hours'] = (hd_stats['Total_Duration_Seconds'] / 3600).round(2)
hd_stats = hd_stats.drop(columns=['Total_Duration_Seconds'])

print("\nHD vs SD Statistics:")
print(hd_stats)


hd_by_program = df.groupby(['program_class', 'quality_type']).agg({
    'user_id_maped': 'nunique',
    'duration_seconds': 'sum'
}).reset_index()

hd_by_program['Total_Duration_Hours'] = (hd_by_program['duration_seconds'] / 3600).round(2)
hd_by_program = hd_by_program.drop(columns=['duration_seconds'])
hd_by_program.columns = ['Program_Class', 'Quality', 'Unique_Users', 'Total_Duration_Hours']

print("\nHD/SD Usage by Program Class:")
print(hd_by_program)


quality_percentage = df['quality_type'].value_counts(normalize=True) * 100
print("\nQuality Distribution:")
for quality, percentage in quality_percentage.items():
    print(f"{quality}: {percentage:.2f}%")


cross_tab = pd.crosstab(df['program_class'], df['quality_type'], normalize='index') * 100
print("\nQuality Preference by Program Class (%):")
print(cross_tab.round(2))


HD vs SD Statistics:
  Quality  Unique_Users  Total_Views  Avg_Duration_Seconds  \
0      HD         11000       405036            801.507069   
1      SD          6728       643539           1501.248164   

   Total_Duration_Hours  
0              90177.56  
1             268364.37  

HD/SD Usage by Program Class:
     Program_Class Quality  Unique_Users  Total_Duration_Hours
0            MOVIE      HD         10880              64856.37
1            MOVIE      SD          6093              38587.78
2  SERIES/EPISODES      HD          2625              25321.19
3  SERIES/EPISODES      SD          3282             229776.59

Quality Distribution:
SD: 61.37%
HD: 38.63%

Quality Preference by Program Class (%):
quality_type        HD     SD
program_class                
MOVIE            67.92  32.08
SERIES/EPISODES  13.08  86.92


In [45]:
fig1 = px.pie(hd_stats, values='Total_Views', names='Quality',
             title='HD vs SD Distribution by Total Views')
fig1.show()

In [46]:
fig2 = px.pie(hd_stats, values='Total_Duration_Hours', names='Quality',
             title='HD vs SD Distribution by Total Watch Time')
fig2.show()

In [47]:
fig3 = px.bar(hd_by_program, x='Program_Class', y='Total_Duration_Hours',
             color='Quality', title='HD vs SD Usage by Program Class',
             barmode='group')
fig3.show()

In [48]:
fig4 = px.bar(cross_tab.reset_index(), x='program_class',
             y=['HD', 'SD'], title='Quality Preference by Program Class (%)',
             barmode='group')
fig4.show()

In [49]:
user_behavior = df.groupby('user_id_maped').agg({
    'hd': 'mean',
    'duration_seconds': 'sum',
    'program_name': 'count'
}).reset_index()

user_behavior.columns = ['User_ID', 'HD_Preference_Rate', 'Total_Duration', 'Total_Views']
user_behavior['Avg_Session_Duration'] = user_behavior['Total_Duration'] / user_behavior['Total_Views']

user_behavior['User_Type'] = user_behavior['HD_Preference_Rate'].apply(
    lambda x: 'HD_User' if x > 0.5 else 'SD_User' if x < 0.5 else 'Mixed_User'
)

user_type_summary = user_behavior['User_Type'].value_counts()
print("\nUser Classification by HD Preference:")
print(user_type_summary)
print(f"\nPercentage distribution:")
for user_type, count in user_type_summary.items():
    percentage = (count / len(user_behavior)) * 100
    print(f"{user_type}: {percentage:.2f}%")


User Classification by HD Preference:
User_Type
HD_User       8064
SD_User       3208
Mixed_User     306
Name: count, dtype: int64

Percentage distribution:
HD_User: 69.65%
SD_User: 27.71%
Mixed_User: 2.64%


In [50]:
total_hd_views = len(df[df['hd'] == 1])
total_views = len(df)
hd_percentage = (total_hd_views / total_views) * 100

print(f"1. Overall HD usage: {hd_percentage:.2f}% of total views")

movies_hd = df[df['program_class'] == 'MOVIE']['hd'].mean() * 100
series_hd = df[df['program_class'] == 'SERIES/EPISODES']['hd'].mean() * 100

print(f"2. HD preference by content type:")
print(f"   - Movies: {movies_hd:.2f}% HD usage")
print(f"   - Series/Episodes: {series_hd:.2f}% HD usage")

hd_avg_duration = df[df['hd'] == 1]['duration_seconds'].mean() / 60
sd_avg_duration = df[df['hd'] == 0]['duration_seconds'].mean() / 60

print(f"3. Average session duration:")
print(f"   - HD viewers: {hd_avg_duration:.2f} minutes")
print(f"   - SD viewers: {sd_avg_duration:.2f} minutes")

1. Overall HD usage: 38.63% of total views
2. HD preference by content type:
   - Movies: 67.92% HD usage
   - Series/Episodes: 13.08% HD usage
3. Average session duration:
   - HD viewers: 13.36 minutes
   - SD viewers: 25.02 minutes
